<a href="https://colab.research.google.com/github/linyuehzzz/5526_neural_networks/blob/master/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Lab 1**
This code implements a two-layer perceptron with the backpropagation algorithm to solve the parity problem.  
Yue Lin (lin.3326 at osu.edu)  
Created: 9/22/2020

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#### **Set up libraries**

In [ ]:
from random import seed, uniform

#### **Generate training data**

#### **Initialize network**

In [ ]:
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[uniform(-1, 1) for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[uniform(-1, 1) for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
seed(1)
network = initialize_network(4, 4, 1)
for layer in network:
	print(layer)